In [ ]:
!pip install spacy -U
!pip install spacy-transformers
!python -m spacy download en_core_web_lg
!pip install pyproject.toml-based
!pip install CuPy

In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.2                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_lg (3.7.1), en_core_web_sm (3.7.1)



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
nlp = spacy.blank("en")

In [5]:
f = open('/content/drive/MyDrive/4th_Year/DSC4996/Spacy/train_data.json')
train_data = json.load(f)
train_data

{'classes': ['POLICY_NO',
  'DATE',
  'DOCTOR',
  'HOSPITAL',
  'AMOUNT',
  'ADDITIONAL ACCIDENT COVER',
  'ADDITIONAL CRITICAL ILLNESS COVER',
  'CASHLESS HOSPITAL COVER',
  'CRITICAL ILLNESS COVER',
  'FAMILY INCOME BEENEFIT',
  'FUNERAL EXPENSES',
  'PERSONAL ACCIDENT COVER'],
 'annotations': [["jxg janashakthi group claims action sheet policy number: nju32014-182 date: 22-02-2023 doctor name: dr anomila hospital name: new pioneer hospital description of accident: patient fell while climbing stairs at home, fracturing their wrist. description of claim: hospital room and board charges for 2 days of inpatient stay, surgeon's fees for wrist fracture repair, anesthesia costs during surgery, direct settlement of eligible medical expenses with the hospital, without upfront payment by the patient. amount (rs.): 20000",
   {'entities': [[57, 69, 'POLICY_NO'],
     [76, 86, 'DATE'],
     [100, 110, 'DOCTOR'],
     [126, 146, 'HOSPITAL'],
     [519, 524, 'AMOUNT'],
     [262, 275, 'CASHLESS H

In [9]:
db = DocBin()

for text, annot in tqdm(train_data['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    doc.ents = ents
    db.add(doc)

db.to_disk("/content/drive/MyDrive/4th_Year/DSC4996/Spacy/train_data.spacy")

100%|██████████| 45/45 [00:00<00:00, 884.57it/s]


In [10]:
!python -m spacy init fill-config /content/drive/MyDrive/4th_Year/DSC4996/Spacy/base_config.cfg /content/drive/MyDrive/4th_Year/DSC4996/Spacy/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/4th_Year/DSC4996/Spacy/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy train /content/drive/MyDrive/4th_Year/DSC4996/Spacy/config.cfg --output /content/drive/MyDrive/4th_Year/DSC4996/Spacy/ --paths.train /content/drive/MyDrive/4th_Year/DSC4996/Spacy/train_data.spacy --paths.dev /content/drive/MyDrive/4th_Year/DSC4996/Spacy/train_data.spacy --gpu-id 0

ℹ Saving to output directory:
/content/drive/MyDrive/4th_Year/DSC4996/Spacy
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
config.json: 100% 481/481 [00:00<00:00, 2.79MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 2.72MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 1.88MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 3.29MB/s]
model.safetensors: 100% 499M/499M [00:01<00:00, 329MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ---

In [18]:
nlp_ner = spacy.load("/content/drive/MyDrive/4th_Year/DSC4996/Spacy/model-best")

In [25]:
input = "JXG JANASHAKTHI GROUP Claims Action Sheet Policy Number: LI4233888 Date: 22-07-2023 Doctor Name: DR.LALITH Hospital Name: RUHUNU HOSPITAL Description of Accident: Paralysis due to a spinal cord injury in a car accident Description of Claim: Claim covers for initial ER treatment, diagnostic tests, ambulance fees, surgery costs, long-term rehabilitation, home modifications Amount (Rs.): 20800"
doc = nlp_ner(input)
spacy.displacy.render(doc, style="ent", jupyter=True)